# Poisoning Network Flow Classifiers from ACSAS 2023

## Install necessary libraries

In [7]:
!pip install --upgrade mergetb
!pip install timeout-timer
!pip install mercury

## Import necessary libraries

In [9]:
import time, socket
from timeout_timer import timeout, TimeoutInterrupt
import subprocess
import os
from mergetb.workspace import User, Experiment, Project
from mergetb.realize import Realization
from mergetb.materialize import Materialization
from mergetb.grpc_client import MergeGRPCError
from mergetb.types import StatusType

## Log in as user

In [10]:
username='marine'
passw='0m@rine1!!'

try:
    u = User(username, passw)
    u.login()
except MergeGRPCError as e:
    print(f"Error logging in: {e}")
else:
    print("Logged in as user", u.username)

Logged in as user marine


## Find or create experiment

In [11]:
try:
    # load the user's personal project
    p = Project(username)
    resp = p.get()

    # look for the experiment, create if it does not exist
    for exp in resp.project.experiments:
        if exp == 'poisnetx':
            print('experiment poisnetx already exists')
            break
    else:
        e = Experiment('poisnetx', username)
        e.create()
        print('experiment poisnetx created')
    
except MergeGRPCError as e:
    print(e)
#proj = m.find_project(username)
#exp = m.find_experiment('hades', proj.name)
#if exp == None:
#    exp = m.create_experiment('hades', proj.name)

experiment poisnetx already exists


## Push model, allocate resources and attach the experiment to the XDC

In [ ]:
RLZ_TIMEOUT=15
MTZ_TIMEOUT=800

try:
    exp = Experiment('poisnetx', username)
    rev = exp.push_model('poisnet.model').revision
    print("Pushed experiment revision", rev)
    time.sleep(5)
    
    # reserve resources through a lease named 'v1' (name is arbitrary)
    exp.reserve('vx', revision=rev)

    # wait up for 10 sec for reservation to succeed
    try:
        rlz = Realization('vx', exp.name, exp.project)
        with timeout(RLZ_TIMEOUT):
            while True:
                resp = rlz.get_status()
                status = resp.status.highest_status

                if status == StatusType.Success:
                    print('leased resources')
                    break
    except TimeoutInterrupt:
        raise Exception('reservation did not succeed within %d seconds' % RLZ_TIMEOUT)

    # activate experiment on this lease
    exp.activate('vx')

    # wait up to 60 sec for activation to succeed
    try:
        mtz = Materialization('vx', exp.name, exp.project)
        with timeout(MTZ_TIMEOUT):
            while True:
                resp = mtz.get()
                status = resp.status.highest_status

                if status == StatusType.Success:
                    print('started experiment')
                    break
    except TimeoutInterrupt:
        raise Exception('activation did not succeed within %d seconds' % MTZ_TIMEOUT)

    hostname = socket.gethostname().split('-')[0].split('.')[0]
    exp.attach_xdc('vx', hostname, username)
    time.sleep(15)
    print("Attached experiment is now running")
except MergeGRPCError as e:
    print(e)

## Run commands in experiment to clone the artifact, install prerequisites, fix some bugs and execute one script in the artifact¶

In [440]:
exp = Experiment('poisnetx', username)
def print_lines(label, lines, elines):
    print(f"--- {label} ---")
    for line in lines:
        print(line.strip())
    for eline in elines:
        print(eline.strip())
   
installpackages = "sudo apt-get update && sudo apt install git -y && sudo ln -s /usr/bin/python3 /usr/bin/python"

clonerepo = "git clone https://github.com/ClonedOne/poisoning_network_flow_classifiers.git"

installotherpackages = "sudo apt install python3-pip -y  "
installotherpackages2 = "source ~/.bashrc"
installotherpackages3 = "pip install gdown"
installotherpackages4 = "pip install tensorflow==2.11.0"
installotherpackages5 = "source ~/.bashrc"
installotherpackages6 = "cd poisoning_network_flow_classifiers; pwd"
installotherpackages7 = "export PATH=$PATH:/home/marine/.local/bin; pip install -r /home/marine/poisoning_network_flow_classifiers/requirements.txt"
installotherpackages8 = "export PATH=$PATH:/home/marine/.local/bin; pip install -e  /home/marine/poisoning_network_flow_classifiers"
installotherpackages9 = "export PATH=$PATH:/home/marine/.local/bin; gdown https://drive.google.com/uc?id=1Q0R9LZr9-4CachGivhfQac_oLnc57kZo -O /home/marine/poisoning_network_flow_classifiers/"
installotherpackages10 = "export PATH=$PATH:/home/marine/.local/bin; cd /home/marine/poisoning_network_flow_classifiers/poisnet; ls"

file_path = "/home/marine/poisoning_network_flow_classifiers/poisnet/constants.py"
old_base_path = "/net/data/netpois/data_out"
new_base_path = "/home/marine/poisoning_network_flow_classifiers/data_out"
modifyconstant4 = f'sed -i "s|{old_base_path}|{new_base_path}|g" {file_path}'

decompressdata = "cd .. && tar -xzvf  /home/marine/poisoning_network_flow_classifiers/netpois_data.tar.gz -C /home/marine/poisoning_network_flow_classifiers"

runcode_figure3 = "cd poisoning_network_flow_classifiers; pwd; bash scripts/bash/entropy_full.sh & bash scripts/bash/gini_full.sh & bash scripts/bash/shap_full.sh & scripts/bash/random_full.sh; pwd"
runcode_figure4= "cd poisoning_network_flow_classifiers; bash scripts/bash/entropy_gen.sh & bash scripts/bash/entropy_red.sh; pwd"
runcode_figure5="cd poisoning_network_flow_classifiers; bash scripts/bash/shap_gen.sh; pwd"


print("Installing packages, this may take a while...")
lines, elines = exp.exec_on_node(username, "nodepoisnet", installpackages)
print_lines("Installing packages", lines, elines)

print("Cloning repository")
lines, elines = exp.exec_on_node(username, "nodepoisnet", clonerepo)
print_lines("Installing packages", lines, elines)

print("Installing some more packages")
lines, elines = exp.exec_on_node(username, "nodepoisnet", installotherpackages)
print_lines("Installing more packages", lines, elines)

lines, elines = exp.exec_on_node(username, "nodepoisnet", installotherpackages2)
print_lines("Installing more packages 2", lines, elines)

lines, elines = exp.exec_on_node(username, "nodepoisnet", installotherpackages3)
print_lines("Installing more packages 3", lines, elines)

lines, elines = exp.exec_on_node(username, "nodepoisnet", installotherpackages4)
print_lines("Installing more packages 4", lines, elines)

lines, elines = exp.exec_on_node(username, "nodepoisnet", installotherpackages5)
print_lines("Installing more packages 5", lines, elines)

lines, elines = exp.exec_on_node(username, "nodepoisnet", installotherpackages6)
print_lines("Installing more packages 6", lines, elines)

lines, elines = exp.exec_on_node(username, "nodepoisnet", installotherpackages7)
print_lines("Installing more packages 7", lines, elines)

elines = exp.exec_on_node(username, "nodepoisnet", installotherpackages8)
print_lines("Installing more packages 8", lines, elines)

lines, elines = exp.exec_on_node(username, "nodepoisnet", installotherpackages9)
print_lines("Installing more packages 9", lines, elines)

lines, elines = exp.exec_on_node(username, "nodepoisnet", installotherpackages10)
print_lines("Installing more packages 10", lines, elines)

print("Modifying constants file")
lines, elines = exp.exec_on_node(username, "nodepoisnet", modifyconstant4)
print_lines("Modifying constants file 4", lines, elines)

print("Decompressing data")
lines, elines = exp.exec_on_node(username, "nodepoisnet", decompressdata)
print_lines("Installing more packages", lines, elines)

print("run code, this will take several hours, up to 50 hours")
print("run code figure 3")
lines, elines = exp.exec_on_node(username, "nodepoisnet", runcode_figure3)

print("run code figure 4")
lines, elines = exp.exec_on_node(username, "nodepoisnet", runcode_figure4)

print("run code figure 5")
lines, elines = exp.exec_on_node(username, "nodepoisnet", runcode_figure5)


Cloning repository
run code figure 6


KeyboardInterrupt: 

## Display results figure 3 and 4

In [407]:
exp = Experiment('poisnetx', username)
def print_lines(label, lines, elines):
    print(f"--- {label} ---")
    for line in lines:
        print(line.strip())
    for eline in elines:
        print(eline.strip())

displayresults_3_4_1="cd ~/poisoning_network_flow_classifiers/notebooks/visualizations; pwd"

displayresults_3_4_2 = """
echo "import nbformat" > /home/marine/poisoning_network_flow_classifiers/notebooks/visualizations/extract.py
echo "import argparse" >> /home/marine/poisoning_network_flow_classifiers/notebooks/visualizations/extract.py
echo "" >> /home/marine/poisoning_network_flow_classifiers/notebooks/visualizations/extract.py
echo "def extract_code_from_ipynb(ipynb_file, output_file):" >> /home/marine/poisoning_network_flow_classifiers/notebooks/visualizations/extract.py
echo "    with open(ipynb_file, 'r', encoding='utf-8') as file:" >> /home/marine/poisoning_network_flow_classifiers/notebooks/visualizations/extract.py
echo "        notebook = nbformat.read(file, as_version=4)" >> /home/marine/poisoning_network_flow_classifiers/notebooks/visualizations/extract.py
echo "" >> /home/marine/poisoning_network_flow_classifiers/notebooks/visualizations/extract.py
echo "    code_cells = [cell['source'] for cell in notebook['cells'] if cell['cell_type'] == 'code']" >> /home/marine/poisoning_network_flow_classifiers/notebooks/visualizations/extract.py
echo "" >> /home/marine/poisoning_network_flow_classifiers/notebooks/visualizations/extract.py
echo "    with open(output_file, 'w', encoding='utf-8') as file:" >> /home/marine/poisoning_network_flow_classifiers/notebooks/visualizations/extract.py
echo "        for i, code in enumerate(code_cells, 1):" >> /home/marine/poisoning_network_flow_classifiers/notebooks/visualizations/extract.py
echo "            file.write(code)" >> /home/marine/poisoning_network_flow_classifiers/notebooks/visualizations/extract.py
echo "            file.write('\\\\n\\\\n')" >> /home/marine/poisoning_network_flow_classifiers/notebooks/visualizations/extract.py
echo "" >> /home/marine/poisoning_network_flow_classifiers/notebooks/visualizations/extract.py
echo "if __name__ == '__main__':" >> /home/marine/poisoning_network_flow_classifiers/notebooks/visualizations/extract.py
echo "    parser = argparse.ArgumentParser(description='Extract code from Jupyter Notebook (.ipynb) file.')" >> /home/marine/poisoning_network_flow_classifiers/notebooks/visualizations/extract.py
echo "    parser.add_argument('ipynb_file', help='The input .ipynb file')" >> /home/marine/poisoning_network_flow_classifiers/notebooks/visualizations/extract.py
echo "    parser.add_argument('output_file', help='The output file to save the extracted code')" >> /home/marine/poisoning_network_flow_classifiers/notebooks/visualizations/extract.py
echo "" >> /home/marine/poisoning_network_flow_classifiers/notebooks/visualizations/extract.py
echo "    args = parser.parse_args()" >> /home/marine/poisoning_network_flow_classifiers/notebooks/visualizations/extract.py
echo "    extract_code_from_ipynb(args.ipynb_file, args.output_file)" >> /home/marine/poisoning_network_flow_classifiers/notebooks/visualizations/extract.py
"""

displayresults_3_4_3_bis_1="pwd"
displayresults_3_4_3_bis_2="cd ~/poisoning_network_flow_classifiers/notebooks/visualizations; ls"
displayresults_3_4_3_bis_3="cd ~/poisoning_network_flow_classifiers/notebooks/visualizations; python3 extract.py ctu13_results.ipynb ctu13_results.py; pwd; ls "
displayresults_3_4_3_bis_4= "cd ~/poisoning_network_flow_classifiers/notebooks/visualizations; sed -i '1,2d' ctu13_results.py"
displayresults_3_4_3_bis_5= "cd ~/poisoning_network_flow_classifiers/notebooks/visualizations; sed -i '/display/s/^/#/' ctu13_results.py"
displayresults_3_4_3_bis_6= "cd ~/poisoning_network_flow_classifiers/notebooks/visualizations; sed -i '/assert False/d' ctu13_results.py" 
displayresults_3_4_3_bis_7= """cd ~/poisoning_network_flow_classifiers/notebooks/visualizations; sed -i 's|base_path = ""|base_path = "data_out"|g' ctu13_results.py""" 
display_results_3_4_4="""cd ~/poisoning_network_flow_classifiers/notebooks/visualizations; sed -i '35,90d' ctu13_results.py"""

print("Extract python code from notebook")

lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_3_4_1)
print_lines("Go in right directory", lines, elines)

lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_3_4_2)
print_lines("Extract python", lines, elines)

lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_3_4_3_bis_1)
print(lines, elines)

lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_3_4_3_bis_2)
print(lines, elines)

lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_3_4_3_bis_3)
print(lines, elines)

lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_3_4_3_bis_4)
print(lines, elines)

lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_3_4_3_bis_5)
print(lines, elines)

lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_3_4_3_bis_6)
print(lines, elines)

lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_3_4_3_bis_7)
print(lines, elines)

lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_3_4_4)
print(lines, elines)


Extract python code from notebook
--- Go in right directory ---
/home/marine/poisoning_network_flow_classifiers/notebooks/visualizations
--- Extract python ---
['/home/marine\n'] []
['ctu13_ae_results.ipynb\n', 'ctu13_results.ipynb\n', 'extract.py\n', 'function_1.py\n', 'function_2.py\n', 'modif.py\n', 'other_results.ipynb\n', 'poisoning_divergence_benign.ipynb\n', 'saved_files\n'] []
['/home/marine/poisoning_network_flow_classifiers/notebooks/visualizations\n', 'ctu13_ae_results.ipynb\n', 'ctu13_results.ipynb\n', 'ctu13_results.py\n', 'extract.py\n', 'function_1.py\n', 'function_2.py\n', 'modif.py\n', 'other_results.ipynb\n', 'poisoning_divergence_benign.ipynb\n', 'saved_files\n'] []
[] []
[] []
[] []
[] []
[] ["bash: -c: line 16: syntax error near unexpected token `('\n", 'bash: -c: line 16: `    x_ticks = sorted(cur_df["Poison percentage (%)"].unique())\'\n']


In [408]:
script = """
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import os

# Initialiser un compteur global pour éviter d’écraser les fichiers
plot_counter = 1

def plot_poisoning_results_line(cur_df, no_eb=False, ds="neris"):
    global plot_counter
    trigger_types = sorted(cur_df["trigger_type"].unique())
    model_type = cur_df["model"].unique()
    assert len(model_type) == 1
    model_type = model_type[0]
    x_ticks = sorted(cur_df["Poison percentage (%)"].unique())

    if no_eb:
        eb = None
    else:
        eb = "se"

    # Créez un fichier PDF pour sauvegarder les figures
    pdf_filename = f"figures_plot_poisoning_results_line_{ds}_{plot_counter}.pdf"
    with PdfPages(pdf_filename) as pdf:
        for trig_type in trigger_types:
            plt.figure(figsize=(9, 5))
            lp = sns.lineplot(
                x="Poison percentage (%)",
                y="poison_asr_poison",
                hue="fstrat",
                data=cur_df[
                    (cur_df["trigger_type"] == trig_type) & 
                    (cur_df["Poison percentage (%)"] != 10.0)
                ],
                alpha=0.9,
                marker="o",
                errorbar=eb,
                linewidth=1.5,
                hue_order=["entropy", "gini", "shap", "random"],
                estimator="mean",
                palette="rocket_r"
            )
            
            lp.set_ylim(-0.02, 1.02)
            lp.set_xticks(x_ticks)
            lp.set_ylabel("Poison ASR")
            leg = lp.get_legend()
            leg.set_title(None)
            texts_to_change = [readable[t.get_text()] for t in leg.texts]
            for t, l in zip(leg.texts, texts_to_change):
                t.set_text(l)
            sns.move_legend(lp, "lower center", bbox_to_anchor=(0.5, -0.5), ncol=4, frameon=False)
            lp.set_title(f"{model_type} - {readable[ds]} - {readable[trig_type]}")
            plt.tight_layout()
            pdf.savefig()
            plt.close()
    plot_counter += 1
"""

with open("function_1.py", "w") as f:
    f.write(script)

In [409]:
import subprocess
username="marine"
process = subprocess.run("scp -o stricthostkeychecking=no -i /home/" + username + "/.ssh/merge_key "  "/project/marine/notebooks/function_1.py marine@nodepoisnet:/home/marine/poisoning_network_flow_classifiers/notebooks/visualizations &> /dev/null", shell=True, stdout=subprocess.DEVNULL)

In [410]:
command= "cd ~/poisoning_network_flow_classifiers/notebooks/visualizations; cat function_1.py >> ctu13_results.py"
lines, elines = exp.exec_on_node(username, "nodepoisnet", command)
print(lines, elines)

[] []


In [411]:
script_1 = """
plot_counter_bar = 1

def plot_poisoning_results_bar(cur_df, no_eb=False, ds="neris"):
    global plot_counter_bar
    trigger_types = sorted(cur_df["trigger_type"].unique())
    model_type = cur_df["model"].unique()
    assert len(model_type) == 1
    model_type = model_type[0]

    pdf_filename = f"figures_plot_poisoning_results_bar_{ds}_{plot_counter_bar}.pdf"
    with PdfPages(pdf_filename) as pdf:
        for trig_type in trigger_types:
            plt.figure(figsize=(9, 5))

            if no_eb:
                eb = None
            else:
                eb = ("se")

            lp = sns.barplot(
                x="Poison percentage (%)",
                y="poison_asr_poison",
                hue="fstrat",
                data=cur_df[((cur_df["trigger_type"] == trig_type) & (cur_df["Poison percentage (%)"] != 10.0))],
                alpha=0.9,
                errorbar=eb,
                hue_order=["entropy", "gini", "shap", "random"],
                estimator=np.mean,
                errwidth=1.5,
                palette="rocket_r"
            )

            lp.set_ylim(-0.02, 1.02)
            lp.set_ylabel("Poison ASR")

            leg = lp.get_legend()
            leg.set_title(None)
            texts_to_change = [readable[t.get_text()] for t in leg.texts]
            for t, l in zip(leg.texts, texts_to_change):
                t.set_text(l)

            sns.move_legend(lp, "lower center", bbox_to_anchor=(0.5, -0.4), ncol=4, frameon=False)

            lp.set_title(f"{model_type} - {readable[ds]} - {readable[trig_type]}")

            sns.despine(left=True, right=True)
            plt.tight_layout()

            pdf.savefig()
            plt.close()

    plot_counter_bar += 1
"""
with open("function_2.py", "w") as f:
    f.write(script_1)


In [412]:
import subprocess
username="marine"
process = subprocess.run("scp -o stricthostkeychecking=no -i /home/" + username + "/.ssh/merge_key "  "/project/marine/notebooks/function_2.py marine@nodepoisnet:/home/marine/poisoning_network_flow_classifiers/notebooks/visualizations &> /dev/null", shell=True, stdout=subprocess.DEVNULL)

In [413]:
command= "cd ~/poisoning_network_flow_classifiers/notebooks/visualizations; cat function_2.py >> ctu13_results.py"
lines, elines = exp.exec_on_node(username, "nodepoisnet", command)
print(lines, elines)

[] []


In [414]:
def print_lines(label, lines, elines):
    print(f"--- {label} ---")
    for line in lines:
        print(line.strip())
    for eline in elines:
        print(eline.strip())

displayresults_3_4_6 = """cd ~/poisoning_network_flow_classifiers/notebooks/visualizations; sed -i '37,91d' ctu13_results.py"""
displayresults_3_4_8="""cd ~/poisoning_network_flow_classifiers/notebooks/visualizations; sed -i '173i\plt.savefig(f"figure_comparison_1.pdf", bbox_inches="tight")' ctu13_results.py"""
displayresults_3_4_8_bis_1="""cd ~/poisoning_network_flow_classifiers/notebooks/visualizations;sed -i '180i\plt.savefig(f"figure_comparison_2.pdf", bbox_inches="tight")' ctu13_results.py"""
displayresults_3_4_8_bis_2="""cd ~/poisoning_network_flow_classifiers/notebooks/visualizations;sed -i '187i\plt.savefig(f"figure_comparison_3.pdf", bbox_inches="tight")' ctu13_results.py"""
displayresults_3_4_8_bis_3="""cd ~/poisoning_network_flow_classifiers/notebooks/visualizations;sed -i '272i\plt.savefig(f"figure_comparison_trigger.pdf", bbox_inches="tight")' ctu13_results.py"""
displayresults_3_4_8_bis_4="""cd ~/poisoning_network_flow_classifiers/notebooks/visualizations; sed -i '322i\plt.savefig(f"figure_last.pdf", bbox_inches="tight")' ctu13_results.py"""
displayresults_3_4_9="mv ~/poisoning_network_flow_classifiers/notebooks/visualizations/ctu13_results.py ~/poisoning_network_flow_classifiers"
displayresults_3_4_9_1="""cd ~/poisoning_network_flow_classifiers/notebooks/visualizations; sed -i '150,151d' ctu13_results.py"""
displayresults_3_4_9_2="""cd ~/poisoning_network_flow_classifiers/notebooks/visualizations; sed -i '163,164d' ctu13_results.py"""

print("Make corrections in python script")

lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_3_4_6)
print(lines, elines)

lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_3_4_8)
print(lines, elines)

lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_3_4_8_bis_1)
print(lines, elines)

lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_3_4_8_bis_2)
print(lines, elines)
lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_3_4_8_bis_3)
print(lines, elines)

lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_3_4_8_bis_4)
print(lines, elines)







Make corrections in python script
[] []
[] []
[] []
[] []
[] []
[] []


In [415]:
script_2="""
plot_poisoning_results_line(neris_results_gb_1.copy(), no_eb=True, ds="neris")
plot_poisoning_results_bar(neris_results_gb_1.copy(), no_eb=False, ds="neris")
plot_poisoning_results_line(neris_results_fn_1.copy(), no_eb=True, ds="neris")
plot_poisoning_results_bar(neris_results_fn_1.copy(), no_eb=False, ds="neris")
"""
with open("modif.py", "w") as f:
    f.write(script_2)

In [416]:
import subprocess
username="marine"
process = subprocess.run("scp -o stricthostkeychecking=no -i /home/" + username + "/.ssh/merge_key "  "/project/marine/notebooks/modif.py marine@nodepoisnet:/home/marine/poisoning_network_flow_classifiers/notebooks/visualizations &> /dev/null", shell=True, stdout=subprocess.DEVNULL)

In [417]:
command_1= "cd ~/poisoning_network_flow_classifiers/notebooks/visualizations; cat modif.py >> ctu13_results.py"
lines, elines = exp.exec_on_node(username, "nodepoisnet", command_1)
print(lines, elines)

[] []


In [418]:
print("Change directory")
lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_3_4_9)

print("Last modifications")

lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_3_4_9_1)
print(lines, elines)

lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_3_4_9_2)
print(lines, elines)


Change directory
Last modifications
[] ['bash: -c: line 0: unexpected EOF while looking for matching `"\'\n', 'bash: -c: line 1: syntax error: unexpected end of file\n']
[] ['bash: -c: line 0: unexpected EOF while looking for matching `"\'\n', 'bash: -c: line 1: syntax error: unexpected end of file\n']
Run python script
--- Run python script ---
Found experiments: ['random', 'gini', 'shap', 'entropy']
Subscenario 1: (175, 33)
Number of results by strategy and trigger type:
fstrat   trigger_type
entropy  full            25
gen             25
red             25
gini     full            25
random   full            25
shap     full            25
gen             25
dtype: int64
Found experiments: ['random', 'gini', 'shap', 'entropy']
Subscenario 1: (175, 33)
Number of results by strategy and trigger type:
fstrat   trigger_type
entropy  full            25
gen             25
red             25
gini     full            25
random   full            25
shap     full            25
gen             

In [475]:
script_3= """
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from poisnet import constants, data_utils

# Chargement des résultats
all_res = pd.concat([neris_results_fn_1, neris_results_gb_1])

# Références f1 pour les modèles
ref_f1s = {
    "neris": {1: {"GradientBoosting": 0.9590540256523703, "FFNN": 0.9269868413691503}},
    "cicids_botnet": {
        "botnet": {"GradientBoosting": 0.9936391135214663, "FFNN": 0.9949295626960601}
    },
    "iscx": {
        "chat_video": {
            "GradientBoosting": 0.9006942331155736,
            "FFNN": 0.9187867946007481,
        },
        "file_video": {
            "GradientBoosting": 0.8003810852662347,
            "FFNN": 0.718457563347224,
        },
    },
}

# Groupement des résultats
both_grps = all_res.groupby(by=["scenario", "subscenario", "model", "fstrat", "Poison percentage (%)", "trigger_type"]).agg({"poison_f1_test": ["mean", "std"]})

# Calcul des deltas F1
f1_deltas = []
for row in both_grps.iterrows():
    cur_scenario = row[0][0]
    cur_subscenario = row[0][1]
    cur_model = row[0][2]
    new_f1 = row[1][("poison_f1_test", "mean")]
    old_f1 = ref_f1s[cur_scenario][cur_subscenario][cur_model]
    delta = new_f1 - old_f1
    f1_deltas.append(delta)

# Ajout de la colonne delta F1
both_grps["f1_delta"] = f1_deltas

# Sélection des résultats avec trigger_type "full"
sel_full = both_grps.reset_index()
sel_full = sel_full[sel_full["trigger_type"] == "full"]

# Aplatir sel_full pour un affichage plus simple
sel_full_flat = sel_full.copy()
sel_full_flat.columns = ["_".join(col).strip() for col in sel_full_flat.columns]
sel_full_flat = sel_full_flat.rename(columns={
    "Poison percentage (%)_": "Poison percentage (%)",
    "poison_f1_test_mean": "poison_f1_test",
    "poison_f1_test_std": "poison_f1_test_std",
    "f1_delta_": "f1_delta"
})

# Agrégation finale
aggregated_results = sel_full_flat.groupby(by=["scenario_", "subscenario_", "model_", "fstrat_", "trigger_type_"]).agg({"f1_delta": ["mean"]})

# Création du fichier PDF
with PdfPages('results.pdf') as pdf:
    
    # Tableau complet des résultats
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.axis('tight')
    ax.axis('off')
    table = ax.table(cellText=both_grps.reset_index().values, colLabels=both_grps.reset_index().columns, loc='center')
    pdf.savefig(fig)
    plt.close()
    
    # Tableau des résultats "full"
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.axis('tight')
    ax.axis('off')
    table = ax.table(cellText=sel_full_flat.values, colLabels=sel_full_flat.columns, loc='center')
    pdf.savefig(fig)
    plt.close()

    # Tableau des résultats agrégés
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.axis('tight')
    ax.axis('off')
    table = ax.table(cellText=aggregated_results.reset_index().values, colLabels=aggregated_results.reset_index().columns, loc='center')
    pdf.savefig(fig)
    plt.close()

print("Les résultats ont été enregistrés dans 'results.pdf'")
"""
with open("modification.py", "w") as f:
    f.write(script_3)


In [476]:
import subprocess
username="marine"
process = subprocess.run("scp -o stricthostkeychecking=no -i /home/" + username + "/.ssh/merge_key "  "/project/marine/notebooks/modification.py marine@nodepoisnet:/home/marine/poisoning_network_flow_classifiers/ &> /dev/null", shell=True, stdout=subprocess.DEVNULL)

In [477]:
supress="cd ~/poisoning_network_flow_classifiers/; sed -i '325,388d' ctu13_results.py"

lines, elines = exp.exec_on_node(username, "nodepoisnet", supress)


In [478]:
command_2= "cd ~/poisoning_network_flow_classifiers/; cat modification.py >> ctu13_results.py"
lines, elines = exp.exec_on_node(username, "nodepoisnet", command_2)
print(lines, elines)

[] []


In [14]:
exp = Experiment('poisnetx', username)
displayresults_3_4_10="cd /home/marine/poisoning_network_flow_classifiers; python3 ctu13_results.py"
print("Run python script")
lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_3_4_10)
print_lines("Run python script", lines, elines)


Run python script


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 5: invalid continuation byte

In [419]:
process = subprocess.run("scp -o stricthostkeychecking=no -i /home/" + username + "/.ssh/merge_key " + username + "@nodepoisnet:/home/marine/poisoning_network_flow_classifiers/figure_*.pdf  /project/marine/notebooks &> /dev/null", shell=True, stdout=subprocess.DEVNULL)

In [420]:
process = subprocess.run("scp -o stricthostkeychecking=no -i /home/" + username + "/.ssh/merge_key " + username + "@nodepoisnet:/home/marine/poisoning_network_flow_classifiers/figures_*.pdf  /project/marine/notebooks &> /dev/null", shell=True, stdout=subprocess.DEVNULL)

In [493]:
process = subprocess.run("scp -o stricthostkeychecking=no -i /home/" + username + "/.ssh/merge_key " + username + "@nodepoisnet:/home/marine/poisoning_network_flow_classifiers/results.pdf  /project/marine/notebooks &> /dev/null", shell=True, stdout=subprocess.DEVNULL)

## Display results figure 3

In [426]:
import mercury as mr
mr.PDF("/project/marine/notebooks/figures_plot_poisoning_results_bar_neris_1.pdf")

In [423]:
import mercury as mr
mr.PDF("/project/marine/notebooks/figures_plot_poisoning_results_bar_neris_2.pdf")

## Display results figure 4

In [424]:
import mercury as mr
mr.PDF("/project/marine/notebooks/figure_comparison_trigger.pdf")

In [425]:
import mercury as mr
mr.PDF("/project/marine/notebooks/figure_last.pdf")

## Display Tables

In [494]:
import mercury as mr
mr.PDF("/project/marine/notebooks/results.pdf")

## Display results figure 5

In [499]:
displayresults_5_1="pwd"
displayresults_5_2="cd ~/poisoning_network_flow_classifiers/notebooks/visualizations; ls"
displayresults_5_3="cd ~/poisoning_network_flow_classifiers/notebooks/visualizations; python3 extract.py poisoning_divergence_benign.ipynb poisoning_div.py; pwd; ls "
displayresults_5_4= "cd ~/poisoning_network_flow_classifiers/notebooks/visualizations; sed -i '1,2d' poisoning_div.py"
displayresults_5_5= "cd ~/poisoning_network_flow_classifiers/notebooks/visualizations; sed -i '/display/s/^/#/' poisoning_div.py"
displayresults_5_6= "cd ~/poisoning_network_flow_classifiers/notebooks/visualizations; sed -i '/assert False/d' poisoning_div.py" 
displayresults_5_7= """cd ~/poisoning_network_flow_classifiers/notebooks/visualizations; sed -i 's|base_path = ""|base_path = "data_out"|g' poisoning_div.py""" 
displayresults_5_8="mv ~/poisoning_network_flow_classifiers/notebooks/visualizations/poisoning_div.py ~/poisoning_network_flow_classifiers"
displayresults_5_9="""cd ~/poisoning_network_flow_classifiers;sed -i '371i\plt.savefig(f"figure_div_1.pdf", bbox_inches="tight")' poisoning_div.py"""
displayresults_5_10="""cd ~/poisoning_network_flow_classifiers;sed -i '400i\plt.savefig(f"figure_div_2.pdf", bbox_inches="tight")' poisoning_div.py"""
displayresults_5_11="""cd ~/poisoning_network_flow_classifiers;sed -i '549i\plt.savefig(f"figure_div_3.pdf", bbox_inches="tight")' poisoning_div.py"""
displayresults_5_12="""cd ~/poisoning_network_flow_classifiers; sed -i 's/from netpois/from poisnet/g' poisoning_div.py"""
displayresults_5_13="""cp -r ~/poisoning_network_flow_classifiers/notebooks/visualizations/saved_files ~/poisoning_network_flow_classifiers/"""


lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_5_1)
print(lines, elines)

lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_5_2)
print(lines, elines)

lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_5_3)
print(lines, elines)

lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_5_4)
print(lines, elines)

lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_5_5)
print(lines, elines)

lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_5_6)
print(lines, elines)

lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_5_7)
print(lines, elines)

lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_5_8)
print(lines, elines)

lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_5_9)
print(lines, elines)

lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_5_10)
print(lines, elines)

lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_5_11)
print(lines, elines)

lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_5_12)
print(lines, elines)

lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_5_13)
print(lines, elines)






['/home/marine\n'] []
['ctu13_ae_results.ipynb\n', 'ctu13_results.ipynb\n', 'extract.py\n', 'function_1.py\n', 'function_2.py\n', 'modif.py\n', 'other_results.ipynb\n', 'poisoning_divergence_benign.ipynb\n', 'saved_files\n'] []
['/home/marine/poisoning_network_flow_classifiers/notebooks/visualizations\n', 'ctu13_ae_results.ipynb\n', 'ctu13_results.ipynb\n', 'extract.py\n', 'function_1.py\n', 'function_2.py\n', 'modif.py\n', 'other_results.ipynb\n', 'poisoning_divergence_benign.ipynb\n', 'poisoning_div.py\n', 'saved_files\n'] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []
[] []


In [500]:
script_4="""
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Enregistrer `distances_df` dans un fichier PDF
with PdfPages('distances_output.pdf') as pdf:
    fig, ax = plt.subplots(figsize=(10, 6))  # Taille de la figure ajustable
    ax.axis('tight')
    ax.axis('off')
    
    # Convertir le DataFrame en tableau dans la figure
    table = ax.table(cellText=distances_df.values, colLabels=distances_df.columns, cellLoc='center', loc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(8)
    table.scale(1.2, 1.2)  # Ajuster l'échelle pour que le texte s'adapte mieux

    # Sauvegarder la figure dans le PDF
    pdf.savefig(fig)
    plt.close(fig)

print("Le fichier 'distances_output.pdf' a été créé avec succès.")

# Enregistrer `table_df` dans un second fichier PDF
with PdfPages('table_output.pdf') as pdf:
    fig, ax = plt.subplots(figsize=(8, 4))  # Ajuster la taille de la figure
    ax.axis('tight')
    ax.axis('off')
    
    # Convertir le DataFrame en tableau dans la figure
    table = ax.table(cellText=table_df.values, colLabels=table_df.columns, cellLoc='center', loc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(8)
    table.scale(1.2, 1.2)  # Ajuster l'échelle pour que le texte s'adapte mieux

    # Sauvegarder la figure dans le PDF
    pdf.savefig(fig)
    plt.close(fig)

print("Le fichier 'table_output.pdf' a été créé avec succès.")

"""
with open("table5.py", "w") as f:
    f.write(script_4)


In [501]:
import subprocess
username="marine"
process = subprocess.run("scp -o stricthostkeychecking=no -i /home/" + username + "/.ssh/merge_key "  "/project/marine/notebooks/table5.py marine@nodepoisnet:/home/marine/poisoning_network_flow_classifiers/ &> /dev/null", shell=True, stdout=subprocess.DEVNULL)


In [502]:
command_3= "cd ~/poisoning_network_flow_classifiers/; cat table5.py >> poisoning_div.py"
lines, elines = exp.exec_on_node(username, "nodepoisnet", command_3)
print(lines, elines)


[] []


In [ ]:
# The displayment of tables here does not work

In [16]:
exp = Experiment('poisnetx', username)
displayresults_5_14="""cd ~/poisoning_network_flow_classifiers; python3 poisoning_div.py"""
lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_5_14)
print(lines, elines)
print("you may want to force the arrest of the script")

KeyboardInterrupt: 

In [504]:
process = subprocess.run("scp -o stricthostkeychecking=no -i /home/" + username + "/.ssh/merge_key " + username + "@nodepoisnet:/home/marine/poisoning_network_flow_classifiers/figure_div_*.pdf  /project/marine/notebooks &> /dev/null", shell=True, stdout=subprocess.DEVNULL)

In [505]:
import mercury as mr
mr.PDF("/project/marine/notebooks/figure_div_1.pdf")

In [506]:
import mercury as mr
mr.PDF("/project/marine/notebooks/figure_div_2.pdf")

In [507]:
import mercury as mr
mr.PDF("/project/marine/notebooks/figure_div_3.pdf")

## Display tables

In [508]:
process = subprocess.run("scp -o stricthostkeychecking=no -i /home/" + username + "/.ssh/merge_key " + username + "@nodepoisnet:/home/marine/poisoning_network_flow_classifiers/distances_output.pdf  /project/marine/notebooks &> /dev/null", shell=True, stdout=subprocess.DEVNULL)

scp: /home/marine/poisoning_network_flow_classifiers/distances_output.pdf: No such file or directory


In [ ]:
process = subprocess.run("scp -o stricthostkeychecking=no -i /home/" + username + "/.ssh/merge_key " + username + "@nodepoisnet:/home/marine/poisoning_network_flow_classifiers/table_output.pdf  /project/marine/notebooks &> /dev/null", shell=True, stdout=subprocess.DEVNULL)


In [ ]:
import mercury as mr
mr.PDF("/project/marine/notebooks/table_output.pdf)

## Make space on node

In [489]:
compress = """
#!/bin/bash

# Spécifiez le répertoire à partir duquel commencer la compression
dir="ctu13_neris"

# Utilisez find pour localiser tous les fichiers dans le répertoire et ses sous-répertoires
find "$dir" -type f -name '*.gz' -exec gzip {} \\;

# Afficher les fichiers décompressés
echo "Fichiers décompressés :"
find "$dir" -type f ! -name '*.gz'
"""

with open("compress.sh", "w") as f:
    f.write(compress)



In [490]:
import subprocess
username="marine"
process = subprocess.run("scp -o stricthostkeychecking=no -i /home/" + username + "/.ssh/merge_key "  "/project/marine/notebooks/compress.sh marine@nodepoisnet:/home/marine/poisoning_network_flow_classifiers/data_out &> /dev/nul", shell=True, stdout=subprocess.DEVNULL)

In [491]:
exp = Experiment('poisnetx', username)
makespace_1="cd ~/poisoning_network_flow_classifiers/data_out; chmod +x compress.sh"
makespace_2="cd ~/poisoning_network_flow_classifiers/data_out; ./compress.sh"

print("Make space on node, this will take an hour or so ")
lines, elines = exp.exec_on_node(username, "nodepoisnet", makespace_1)
lines, elines = exp.exec_on_node(username, "nodepoisnet", makespace_2)
print("end")


Make space on node, this will take an hour or so 
end


## Run code table 4 and figure 6 and display results figure 6

In [462]:
correctconstants_1="""cd poisoning_network_flow_classifiers/poisnet; sed -i 's|"{base_path}/iscxvpn//NonVPN/file_video/"|"{base_path}/iscxvpn/file_video/"|' constants.py"""
correctconstants_2="""cd poisoning_network_flow_classifiers/poisnet; sed -i 's|"{base_path}/iscxvpn//NonVPN/chat_video/"|"{base_path}/iscxvpn/chat_video/"|' constants.py"""
#lines, elines = exp.exec_on_node(username, "nodepoisnet", correctconstants_1)
#lines, elines = exp.exec_on_node(username, "nodepoisnet", correctconstants_2)

runcode_figure6="cd poisoning_network_flow_classifiers; bash scripts/bash/entropy_full_cicids.sh"
runcode_figure6_1="cd poisoning_network_flow_classifiers; bash scripts/bash/shap_full_ccids.sh" 
runcode_figure6_2="cd poisoning_network_flow_classifiers; bash scripts/bash/entropy_full_iscx.sh"  
runcode_figure6_3="cd poisoning_network_flow_classifiers; bash scripts/bash/shap_full_iscx.sh; pwd"
runcode_table4="cd poisoning_network_flow_classifiers; bash scripts/bash/entropy_ae_multi.sh"



print("run code figure 6")
lines, elines = exp.exec_on_node(username, "nodepoisnet", runcode_figure6)
lines, elines = exp.exec_on_node(username, "nodepoisnet", runcode_figure6_1)
lines, elines = exp.exec_on_node(username, "nodepoisnet", runcode_figure6_2)
lines, elines = exp.exec_on_node(username, "nodepoisnet", runcode_figure6_3)

print("run code table 4")
lines, elines = exp.exec_on_node(username, "nodepoisnet", runcode_table4)
print("End")

run code figure 6
run code table 4
End


In [470]:
displayresults_6="cd ~/poisoning_network_flow_classifiers/notebooks/visualizations; python3 extract.py other_results.ipynb other.py"
lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_6)
display_6="""cd ~/poisoning_network_flow_classifiers/notebooks/visualizations; sed -i 's|"{base_path}/ISCXVPN/NonVPN/file_video/supervised/poisoning/8/GradientBoosting"|"{base_path}/iscxvpn/file_video/supervised/poisoning/8/GradientBoosting"|' other.py"""
lines, elines = exp.exec_on_node(username, "nodepoisnet", display_6)

display_7="""cd ~/poisoning_network_flow_classifiers/notebooks/visualizations; sed -i 's|"{base_path}/ISCXVPN/NonVPN/chat_video/supervised/poisoning/8/GradientBoosting"|"{base_path}/iscxvpn/chat_video/supervised/poisoning/8/GradientBoosting"|' other.py"""
lines, elines = exp.exec_on_node(username, "nodepoisnet", display_7)

display_8="""cd ~/poisoning_network_flow_classifiers/notebooks/visualizations; sed -i 's|"{base_path}/ISCXVPN/NonVPN/file_video/supervised/poisoning/8/FFNN"|"{base_path}/iscxvpn/file_video/supervised/poisoning/8/FFNN"|' other.py"""
lines, elines = exp.exec_on_node(username, "nodepoisnet", display_8)

display_9="""cd ~/poisoning_network_flow_classifiers/notebooks/visualizations; sed -i 's|"{base_path}/ISCXVPN/NonVPN/chat_video/supervised/poisoning/8/FFNN"|"{base_path}/iscxvpn/chat_video/supervised/poisoning/8/FFNN"|' other.py"""
lines, elines = exp.exec_on_node(username, "nodepoisnet", display_9)

displayresults_1= "cd ~/poisoning_network_flow_classifiers/notebooks/visualizations; sed -i '1,2d' other.py"
lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_1)

displayresults_2= "cd ~/poisoning_network_flow_classifiers/notebooks/visualizations; sed -i '/display/s/^/#/' other.py"
lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_2)

displayresults_3= "cd ~/poisoning_network_flow_classifiers/notebooks/visualizations; sed -i '/assert False/d' other.py" 
lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_3)

displayresults_4= """cd ~/poisoning_network_flow_classifiers/notebooks/visualizations; sed -i 's|base_path = ""|base_path = "data_out"|g' other.py""" 
lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_4)

displayresults_5="mv ~/poisoning_network_flow_classifiers/notebooks/visualizations/other.py ~/poisoning_network_flow_classifiers"
lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_5)

displayresults_6_1="""cd ~/poisoning_network_flow_classifiers;sed -i '460i\plt.savefig(f"figure_6_1.pdf", bbox_inches="tight")' other.py"""
lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_6_1)

displayresults_7="""cd ~/poisoning_network_flow_classifiers;sed -i '589i\plt.savefig(f"figure_6_2.pdf", bbox_inches="tight")' other.py"""
lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_7)

displayresults_8="""cd ~/poisoning_network_flow_classifiers;sed -i '718i\plt.savefig(f"figure_6_3.pdf", bbox_inches="tight")' other.py"""
lines, elines = exp.exec_on_node(username, "nodepoisnet", displayresults_8)

results="cd ~/poisoning_network_flow_classifiers; python3 other.py"
lines, elines = exp.exec_on_node(username, "nodepoisnet", results)


In [471]:
process = subprocess.run("scp -o stricthostkeychecking=no -i /home/" + username + "/.ssh/merge_key " + username + "@nodepoisnet:/home/marine/poisoning_network_flow_classifiers/figure_6_*.pdf  /project/marine/notebooks &> /dev/null", shell=True, stdout=subprocess.DEVNULL)

In [472]:
import mercury as mr
mr.PDF("/project/marine/notebooks/figure_6_1.pdf")

In [473]:
import mercury as mr
mr.PDF("/project/marine/notebooks/figure_6_2.pdf")

In [474]:
import mercury as mr
mr.PDF("/project/marine/notebooks/figure_6_3.pdf")

## The user may adapt this narrative to display tables and/or different graphs etc. These changes have to be made in the extract files from the notebook such as other.py, ctu13_results.py or poisoning_div.py

## Return resources and if you so wish delete the experiment

In [ ]:
try:
    exp.relinquish('vx')
except MergeGRPCError as e:
    print(e)
else:
    print("experiment relinquished")
#exp.delete()